### This is a pretty interesting project. It took me around one month to finish. Here I will explain some of the difficulties I met and how they are overcomed. I will also explained how the training model was built and the Keras structure I used for the model.

1. The most difficult thing at the beginning of the project is I have no idea whether it's the problem of my training data or  my Keras model when the car was only running a few seconds and crash on the roadside of the simulator. These problem was overcomed step by step by improving both of the training data part as well as the Keras model part. For this project I have to say, both the feeding data and the Keras architecture are critical to the final result of the model. 

2. I use a regular Toshiba laptop for the training without a Graph card. For models with 100, 000 parameters it took around 20 min to train 10 EPOCHS and for models with 200, 000 parameters it took around one hour. The training process indeed took me a lot of time, but I could also use the training time to do a lot of side work such as designing the next Keras model. 

3. I only use the sample data provided by the Udacity for training. I do have a joystick, and I tried both the stable and Beta simulators, however, finally I gave up abtaining my own training data. The reason is because whenever I use my own training data or I merged my training data with the sample data, the driving model I got would perform worse than when I use the sample data only. It actually took me more than 10 hours of time for obtaining the training data, but I'm not a good game player, I couldn't control the throttle very well, and turning the corners especially the last two in track 1 was painful at such a high speed. I hope in a real self drving car project I don't have to be the driver by myself.

4. I use the sample data only, but of course only the existing sample data are not enough for obtaining a good model as I have tried the sample data over all the model I have designed and it always failed at the last two corners of Track 1. As a result, the side camera images were included in the training data, and serveral image augmentations, such as changing brightness, flipping, adding random shadow and transforming image within certain range were employed to generate more data for training. Please be noted the side camera method and the image augementation method were copied and modified from the webpage https://chatbotslife.com/using-augmentation-to-mimic-human-driving-496b569760a9#.lx3y9nl5a. Details will be explained in the main body.

5. A data generator was used for generating data. The generator keeps generating 32 data randomly from the training data per batch. As the training data contains too many zero steering angles which might cause the car having a tendency of driving straigt forward, a keeper was used to filter out part of the data with low steering angles. The idea of the keeper was also from the webpage https://chatbotslife.com/using-augmentation-to-mimic-human-driving-496b569760a9#.lx3y9nl5a and it was modified to accormodate my own code.

6. The Keras model I used is the same as the Nvidia paper, with ELU added into each layer to introduce nonlinearity into the model. I also tried several other models by adding different layers such as MaxPooling2D, AveragePooling2D. It turns out that most of the model I designed worked well on the first 3/4 portion of the track, but the car just can't turn over the first corner after the bridge. It seems like most model I designed couldn't extact enough features from the images right after the bridge. If we check those images carefully, we could find in that corner the right line break off for several meters, which becomes the biggest obstacle hindering the success of most models. Anyway, after testing all different layers I find the Nvidia model with ELU() layers is efficient in training the data, and finally wil; give the acceptable result. The idea of introducing ELU was from the webpage of https://chatbotslife.com/learning-human-driving-behavior-using-nvidias-neural-network-model-and-image-augmentation-80399360efee#.4pikhnmrs. The data in the model was normalized into (-1, 1) first. 

7. I tried to resize the images into different sizes. My experience about the image size is that larger size images keep more useful informations and are better for feature extraction. For example, the model using resize of 32×64 images can't pass the bridge which might because it didn't extract the feature for the road on the bridge very well. The model using 48×64 images could pass the bridge with no problem, however, it always failed the first corner after the bridge. It seems like the model has difficulties in extracting enough features that the right line was missing in the corner. Finally I choose to use the 66×200 as the image size, which might provide more information for the model to extract. Of course, the model layers has to be adjusted in order to fit different image sizes, thus, changing the image size actually brings more changes to the model than just changing the size only.

In [1]:

import pickle
import numpy as np
import base64
from sklearn.model_selection import train_test_split
import math
import os
import pandas as pd
from scipy import ndimage
import time
import tensorflow as tf
from PIL import Image
import scipy.misc
from io import BytesIO
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2


In [2]:
def augment_brightness_camera_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = 0.25 + np.random.uniform()
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def trans_image(image,steer,trans_range):
    rows, cols = image.shape[:2]
    tr_x = trans_range*np.random.uniform()-trans_range/2
    steer_ang = steer + tr_x*0.0033
    tr_y = 40*np.random.uniform()-40/2
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])
    image_tr = cv2.warpAffine(image,Trans_M,(cols,rows))
    return image_tr,steer_ang

def add_random_shadow(image):
    top_y = 320*np.random.uniform()
    top_x = 0
    bot_x = 160
    bot_y = 320*np.random.uniform()
    image_hls = cv2.cvtColor(image,cv2.COLOR_RGB2HLS)
    shadow_mask = 0*image_hls[:,:,1]
    X_m = np.mgrid[0:image.shape[0],0:image.shape[1]][0]
    Y_m = np.mgrid[0:image.shape[0],0:image.shape[1]][1]
    shadow_mask[((X_m-top_x)*(bot_y-top_y) -(bot_x - top_x)*(Y_m-top_y) >=0)]=1
    
    if np.random.randint(2)==1:
        random_bright = .5
        cond1 = shadow_mask==1
        cond0 = shadow_mask==0
        if np.random.randint(2)==1:
            image_hls[:,:,1][cond1] = image_hls[:,:,1][cond1]*random_bright
        else:
            image_hls[:,:,1][cond0] = image_hls[:,:,1][cond0]*random_bright    
    image = cv2.cvtColor(image_hls,cv2.COLOR_HLS2RGB)
    return image


def preprocessImage(image):
    
    image = image[math.floor(160/5):(160-25), 0:320]
    image = scipy.misc.imresize(image, (66, 200))
 
    return image


In [3]:
root = os.getcwd()
data_folder = os.path.join(root, 'data')
csv_file = os.path.join(root, 'data\driving_log.csv')
data = pd.read_csv(csv_file, usecols = ['center', 'left', 'right', 'steering'])


image = list(zip(list(data.center), list(data.left), list(data.right)))
steering = list(data.steering)

center, steering = shuffle(image, steering)

path_train, path_valid, y_train, y_valid = train_test_split(image, steering, test_size=0.1, random_state=36) 

print(len(y_train))


def gen_single(path, y):
    image_path, steer = path, y
    dice = np.random.randint(3)
    if dice == 0:
        path_file = image_path[0]
        shift_ang = 0
    if dice == 1:
        path_file = image_path[1]
        shift_ang = 0.20
    if dice == 2:
        path_file = image_path[2]
        shift_ang = -0.20
    steer_plus = steer + shift_ang

    image = cv2.imread(path_file)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image,steer_plus = trans_image(image, steer_plus, 100)
    image = augment_brightness_camera_images(image)
    image = add_random_shadow(image)
    image = preprocessImage(image)
    image = np.array(image)
    ind_flip = np.random.randint(2)
    if ind_flip==0:
        image = cv2.flip(image,1)
        steer_plus = -steer_plus
    return image, steer_plus




def generate_arrays(path, y, BATCH_SIZE = 32):
    while 1:
        path, y = shuffle(path, y)
        num_examples = len(y)
        batch_y = np.zeros(BATCH_SIZE)
        batch_X = np.zeros((BATCH_SIZE, 66, 200, 3))

        
        for num in range(BATCH_SIZE):            
            keeper = 0
            while keeper == 0:
                rand_index = np.random.randint(num_examples)
                image, steering = gen_single(path[rand_index],y[rand_index])

                if abs(steering) < 0.06:
                    pr_val = np.random.uniform()
                    if pr_val > 0.4:
                        keeper = 1
                else:
                    keeper = 1
            batch_X[num, :, :, :] = image
            batch_y[num] = steering
        yield(batch_X, batch_y)
                        
        


7232


In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, MaxPooling2D, AveragePooling2D, Dropout, Activation, Dense, Lambda, ELU, BatchNormalization
from keras.regularizers import l2

model = Sequential()
model.add(Lambda(lambda x: x/127.5-1, input_shape = (66, 200, 3)))
model.add(Conv2D(24, 5, 5, subsample=(2, 2),  border_mode='valid', init = 'he_normal'))
model.add(ELU())
model.add(Conv2D(36, 5, 5, subsample=(2, 2),  border_mode='valid', init = 'he_normal'))
model.add(ELU())
model.add(Conv2D(48, 5, 5, subsample=(2, 2),  border_mode='valid', init = 'he_normal'))
model.add(ELU())
model.add(Conv2D(64, 3, 3, subsample=(1, 1),  border_mode='valid', init = 'he_normal'))
model.add(ELU())
model.add(Conv2D(64, 3, 3, subsample=(1, 1),  border_mode='valid', init = 'he_normal'))
model.add(ELU())


model.add(Flatten())
model.add(Dense(100, init = 'he_normal'))
model.add(ELU())
model.add(Dense(50, init = 'he_normal'))
model.add(ELU())
model.add(Dense(10, init = 'he_normal'))
model.add(ELU())
model.add(Dense(1, init = 'he_normal'))


model.summary()




Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 66, 200, 3)    0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 31, 98, 24)    1824        lambda_1[0][0]                   
____________________________________________________________________________________________________
elu_1 (ELU)                      (None, 31, 98, 24)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 14, 47, 36)    21636       elu_1[0][0]                      
___________________________________________________________________________________________

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout, Activation, Dense
from keras.models import model_from_json
from keras.models import load_model

with open('model8.json', 'r') as f:
    json_string = f.read()
f.close()
model = model_from_json(json_string)

try:
    model.load_weights('model8.h5')
except:
    print("Unexpected error")

model.summary()

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 66, 200, 3)    0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 31, 98, 24)    1824        lambda_1[0][0]                   
____________________________________________________________________________________________________
elu_1 (ELU)                      (None, 31, 98, 24)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 14, 47, 36)    21636       elu_1[0][0]                      
___________________________________________________________________________________________

In [5]:
train_generator = generate_arrays(path_train, y_train)
valid_generator = generate_arrays(path_valid, y_valid)

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit_generator(train_generator,
                    samples_per_epoch = (len(y_train)//32)*32, nb_epoch = 8,
                    validation_data = valid_generator,  nb_val_samples=len(y_valid), 
                    )

json_string = model.to_json()

with open('model8.json', 'w') as f:
    f.write(json_string)
f.close()
try:    
    model.save_weights('model8.h5', overwrite = True)
except:
    print("Unexpected error")

Epoch 1/8
7232/7232 [==============================] - 343s - loss: 0.0549 - acc: 0.0000e+00 - val_loss: 0.0506 - val_acc: 0.0000e+00
Epoch 2/8
7232/7232 [==============================] - 244s - loss: 0.0469 - acc: 0.0000e+00 - val_loss: 0.0519 - val_acc: 0.0000e+00
Epoch 3/8
7232/7232 [==============================] - 214s - loss: 0.0452 - acc: 0.0000e+00 - val_loss: 0.0453 - val_acc: 0.0000e+00
Epoch 4/8
7232/7232 [==============================] - 201s - loss: 0.0449 - acc: 0.0000e+00 - val_loss: 0.0453 - val_acc: 0.0000e+00
Epoch 5/8
7232/7232 [==============================] - 191s - loss: 0.0438 - acc: 0.0000e+00 - val_loss: 0.0420 - val_acc: 0.0000e+00
Epoch 6/8
7232/7232 [==============================] - 183s - loss: 0.0420 - acc: 0.0000e+00 - val_loss: 0.0425 - val_acc: 0.0000e+00
Epoch 7/8
7232/7232 [==============================] - 183s - loss: 0.0432 - acc: 0.0000e+00 - val_loss: 0.0387 - val_acc: 0.0000e+00
Epoch 8/8
7232/7232 [==============================] - 190s - 